In [3]:
import os
import pickle

import numpy as np
from sklearn.datasets import load_iris
from sklearn.externals import joblib


model = joblib.load('./ACADEMY.joblib')



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [4]:
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#@title Example form fields
#@markdown Forms support many types of fields.

square_feet = 4300 #@param {type:"slider", min:0, max:10000, step:100}
bedrooms = 7 #@param {type:"slider", min:0, max:8, step:1}
age_of_dwelling = 5 #@param {type:"number"}
full_bathrooms = 1 #@param {type:"slider", min:0, max:6, step:1}
land_size_sq_ft = 13200 #@param {type:"number"}
condo = 
#@markdown ---


In [0]:
%%writefile predictor.py
import os
import pickle

import numpy as np
from sklearn.datasets import load_iris
from sklearn.externals import joblib

class MyPredictor(object):
  def __init__(self, model, preprocessor):
    self._model = model
    self._preprocessor = preprocessor
    self._class_names = load_iris().target_names ## NATE REVIEW

  def predict(self, instances, **kwargs):
    inputs = np.asarray(instances) ## NATE REVIEW
    preprocessed_inputs = self._preprocessor.preprocess(inputs)
    if kwargs.get('probabilities'):
      probabilities = self._model.predict_proba(preprocessed_inputs)
      return probabilities.tolist()
    else:
      outputs = self._model.predict(preprocessed_inputs)
      return [self._class_names[class_num] for class_num in outputs]

  @classmethod
  def from_path(cls, Models):
    model_path = os.path.join(Models, f'{neighborhood}.joblib') ## NATE REVIEW
    model = joblib.load(model_path)

    preprocessor_path = os.path.join(Models, 'preprocessor.pkl') ## NATE REVIEW
    with open(preprocessor_path, 'rb') as f:
      preprocessor = pickle.load(f)

    return cls(model, preprocessor)

In [0]:
%%writefile setup.py
from setuptools import setup

setup(
    name='my_custom_code',
    version='0.1',
    scripts=['predictor.py', 'preprocess.py'])

In [0]:
! python setup.py sdist --formats=gztar

In [0]:
! gsutil cp ./dist/my_custom_code-0.1.tar.gz gs://$denver-neighborhood-model/Models/my_custom_code-0.1.tar.gz ## NATE REVIEW
! gsutil cp model.joblib preprocessor.pkl gs://$denver-neighborhood-model/Models ## NATE REVIEW

In [0]:
MODEL_NAME = 'SalesPredictor'
VERSION_NAME = 'v1'

In [0]:
! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION

In [0]:
# --quiet automatically installs the beta component if it isn't already installed 
! gcloud --quiet beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --origin gs://$denver-neighborhood-model/Models \
  --package-uris gs://$denver-neighborhood-model/Models/my_custom_code-0.1.tar.gz \
  --prediction-class predictor.MyPredictor

In [0]:
! pip install --upgrade google-api-python-client

In [0]:
import googleapiclient.discovery

instances = [
    square_feet, bedrooms, age_of_dwelling, full_bathrooms
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

In [0]:
response = service.projects().predict(
    name=name,
    body={'instances': instances, 'probabilities': True}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

In [0]:
# Delete version resource
! gcloud ai-platform versions delete $VERSION_NAME --quiet --model $MODEL_NAME 

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

# Delete Cloud Storage objects that were created
! gsutil -m rm -r gs://$BUCKET_NAME/custom_prediction_routine_tutorial